In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_10_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240108.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

In [4]:
E5="tests/e5_docs_000_999_50_sampled_queries.tsv"
E5_BASE="tests/e5_base_docs_000_999_50_sampled_queries.tsv"

BM25_1M="tests/bm25_1M_docs_000_999_sampled_50_queries.tsv"
BM25_1M_MASSIVEWEB="tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries.tsv"

SPLADE_MT5="tests/splade_mt5_docs_000_099_50_sampled_queries.tsv"

In [19]:
def prepare_run_format(results_filename, add_index=True, score_field='score', invert_score=False):
    
    results_df = pd.read_csv(results_filename, sep='\t')
    
    display(results_df.head())
    
    results_grouped_df = results_df.groupby("query_id")['passage_id'].count()
    
    how_many_passages_per_question = results_grouped_df.iloc[0]

    if invert_score:
        print("Inverting score values in '{}' field...".format(score_field))
        
        results_df['score_inv'] = 1 - results_df[score_field]
        score_field = 'score_inv' 
    
    run_df = results_df.sort_values(["query_id", score_field], ascending=[True, False])
    
    run_df['Q0'] = "Q0"
    run_df['comment'] = "gpt-4-1106-preview"
    
    if add_index:
        run_df['index'] = np.tile(list(range(1, how_many_passages_per_question + 1)), results_grouped_df.shape[0])
    
    output_filename = "{}_run.txt".format(os.path.splitext(results_filename)[0])
    
    print("output_filename={}".format(output_filename))
    
    run_df[['query_id', 'Q0', "passage_id", "index", score_field, "comment"]].to_csv(output_filename, header=None, index=False, sep=" ")
          
    return output_filename, run_df

In [6]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [24]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [25]:
if os.path.exists(qrels_filename) == False:
    eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

    eval_df

    eval_df.drop_duplicates("query_id")[['query_id', 'query']]

    #### Check scores distribution on the current qrels

    np.unique(eval_df['score'].to_numpy(), return_counts=True)

    for group_name, group_df in eval_df.groupby('query_id'):
        print("\nquery_id={}".format(group_name))

        print(np.unique(group_df['score'].to_numpy(), return_counts=True))

    eval_df.groupby('query_id').count()

    eval_df.groupby('query_id').count()['query'].describe()

    #### Prepare the qrels format for trec_eval tools

    eval_df['0'] = '0'

    qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

    qrels_filename

    eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")
else:
    print("QRELS file ({}) already created...".format(qrels_filename))

QRELS file (tests/evaluation_10_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240108_qrel.txt) already created...


### Compute BM25 nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [ ]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [ ]:
retrievers_results

### Compute BM25 + E5 nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [ ]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [ ]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [ ]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [ ]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare E5 run and compute nDCG@10

In [20]:
results = prepare_run_format(E5, score_field='score', invert_score=True)

,query,passage_id,score,query_id,colbertx_id,passage
0,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-51-03396_2,0.200618,105,2433402,Que cena está representada no fundo? Elabore um texto a respeito das relações entre europeus e africanos na costa atlântica a partir do século XV Anterior Próximo Aproveite o Brainly ao máximo 15 respostas verificadas por especialistas todos os dias Notas mais altas Avalie o seu progresso Cadastre-se grátis Já tem uma conta? Entre aqui
1,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-66-05003_5,0.202220,105,3147750,Encontre as soluções dos livros de Geografia Anúncio Ainda tem perguntas? Encontre mais respostas Faça sua pergunta Anterior Próximo Aproveite o Brainly ao máximo 15 respostas verificadas por especialistas todos os dias Notas mais altas Avalie o seu progresso Cadastre-se grátis Já tem uma conta? Entre aqui
2,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-67-09140_1,0.204239,105,3204566,Gostaria de saber a mudança entre as placas Por 5 horas atrás 0 Livros Cursos Por Gabriel Torres 17 de fevereiro
3,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-61-06354_1,0.204784,105,2896596,Pronto para mergulhar de cabeça e desbloquear o emblema? Faça sua pergunta Novas perguntas de Geografia Anterior Próximo Aproveite o Brainly ao máximo 15 respostas verificadas por especialistas todos os dias Notas mais altas Avalie o seu progresso Cadastre-se grátis Já tem uma conta? Entre aqui
4,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-84-00974_1,0.205789,105,3990264,Ainda tem perguntas? Pronto para mergulhar de cabeça e desbloquear o emblema? Faça sua pergunta Anterior Próximo Aproveite o Brainly ao máximo 15 respostas verificadas por especialistas todos os dias Notas mais altas Avalie o seu progresso Cadastre-se grátis Já tem uma conta? Entre aqui


Inverting score values in 'score' field...
output_filename=tests/e5_docs_000_999_50_sampled_queries_run.txt


In [21]:
results[0]

'tests/e5_docs_000_999_50_sampled_queries_run.txt'

In [26]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {results[0]}

In [ ]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare E5 base run and compute nDCG@10

In [28]:
results = prepare_run_format(E5_BASE, score_field='score_inv', invert_score=False)

,query,passage_id,score,query_id,score_inv,colbertx_id,passage
0,Onde está localizada a Praça XV de Novembro?,clueweb22-pt0000-72-08374_0,0.186375,105,0.813625,3457819,"BBC Brasil: Principais Notícias | Folha Abrir sidebar menu Menu Assine Entrar Entrar Minha Folha Artigos salvos Newsletters Assinatura Editar Perfil Senha e Conta Atendimento Sair Oferta Especial: 3 meses por R$ 1,90 Assine a Folha Últimas notícias Gostaria de receber as principais notícias do Brasil e do mundo? Não, obrigado Sim, aceito Assine Fechar sidebar menu Entrar Minha Folha Artigos salvos Newsletters Assinatura Editar perfil Senha e conta Atendimento Sair Edição Folha Últimas Textos da edição folha opinião Editoriais Charges Tendências/Debates colunas e blogs Colunas Blogs Rotativas mais populares Mais lidas Mais enviadas Mais comentadas acervo folha fotografia política Governo Bolsonaro Eleições 2022 painel Entrevista da 2ª economia Previdência Folhainvest agrofolha painel s.a. Folha ESG startups & fintechs mpme Publicidade Legal Recall cotidiano educação rio de janeiro loterias mundo coreia do norte china governo biden venezuela Onde se fala português saúde equilíbrio ciência covid ambiente esporte campeonato brasileiro Futebol Internacional ilustrada mônica bergamo quadrinhos painel das letras Escuta Aqui É coisa fina Comida ilustríssima independência, 200 podcasts F5 Bichos celebridades horóscopo guia folha turismo folhinha especiais deltafolha web stories dias melhores Seminários Folha empreendedor social tv folha o melhor de sãopaulo parceiros Aeroin"
1,Qual o uso dos códigos SWIFT?,clueweb22-pt0000-72-08374_0,0.172405,154,0.827595,3457819,"BBC Brasil: Principais Notícias | Folha Abrir sidebar menu Menu Assine Entrar Entrar Minha Folha Artigos salvos Newsletters Assinatura Editar Perfil Senha e Conta Atendimento Sair Oferta Especial: 3 meses por R$ 1,90 Assine a Folha Últimas notícias Gostaria de receber as principais notícias do Brasil e do mundo? Não, obrigado Sim, aceito Assine Fechar sidebar menu Entrar Minha Folha Artigos salvos Newsletters Assinatura Editar perfil Senha e conta Atendimento Sair Edição Folha Últimas Textos da edição folha opinião Editoriais Charges Tendências/Debates colunas e blogs Colunas Blogs Rotativas mais populares Mais lidas Mais enviadas Mais comentadas acervo folha fotografia política Governo Bolsonaro Eleições 2022 painel Entrevista da 2ª economia Previdência Folhainvest agrofolha painel s.a. Folha ESG startups & fintechs mpme Publicidade Legal Recall cotidiano educação rio de janeiro loterias mundo coreia do norte china governo biden venezuela Onde se fala português saúde equilíbrio ciência covid ambiente esporte campeonato brasileiro Futebol Internacional ilustrada mônica bergamo quadrinhos painel das letras Escuta Aqui É coisa fina Comida ilustríssima independência, 200 podcasts F5 Bichos celebridades horóscopo guia folha turismo folhinha especiais deltafolha web stories dias melhores Seminários Folha empreendedor social tv folha o melhor de sãopaulo parceiros Aeroin"
2,Qual o nosso atual sistema de governo?,clueweb22-pt0000-72-08374_0,0.168410,21,0.831590,3457819,"BBC Brasil: Principais Notícias | Folha Abrir sidebar menu Menu Assine Entrar Entrar Minha Folha Artigos salvos Newsletters Assinatura Editar Perfil Senha e Conta Atendimento Sair Oferta Especial: 3 meses por R$ 1,90 Assine a Folha Últimas notícias Gostaria de receber as principais notícias do Brasil e do mundo? Não, obrigado Sim, aceito Assine Fechar sidebar menu Entrar Minha Folha Artigos salvos Newsletters Assinatura Editar perfil Senha e conta Atendimento Sair Edição Folha Últimas Textos da edição folha opinião Editoriais Charges Tendências/Debates colunas e blogs Colunas Blogs Rotativas mais populares Mais lidas Mais enviadas Mais comentadas acervo folha fotografia política Governo Bolsonaro Eleições 2022 painel Entrevista da 2ª economia Previdência Folhainvest agrofolha painel s.a. Folha ESG startups & fintechs mpme Publicidade Legal Recall coti

output_filename=tests/e5_base_docs_000_999_50_sampled_queries_run.txt


In [29]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {results[0]}

In [ ]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 1M run and compute nDCG@10

In [31]:
results = prepare_run_format(BM25_1M, score_field='score', invert_score=False)

,query_id,Q0,passage_id,position,score,comment,passage
0,1,Q0,clueweb22-pt0000-79-16312_218,1,9.4348,Anserini,"Fonte: http://www.gazetadopovo.com.br/esportes/olimpiadas/conteudo.phtml?tit=Cameron-aposta-na-Olimpiada-para-recuperar-economia&id=1279366 Tecnológica Titulo: Atletas brasileiros utilizam uniformes tecnológicos na Olimpíada Em busca de medalhas nos Jogos Olímpicos de Londres, os atletas brasileiros terão a tecnologia como um fator aliado. Os representantes nacionais no atletismo e no vôlei tiveram tênis especiais desenvolvidos pela Asics, enquanto os atletas da Confederação Brasileira de Tênis (CBT) e da Confederação Brasileira de Handebol (CBHb) contarão com uniformes oficiais da marca. Criados pela estilista Tatiana Mancini, os novos uniformes foram feitos com tecidos inteligentes e tecnológicos. Além disso, as roupas desenvolvidas por Mancini apresentam características especiais que remetem aos principais aspectos do Brasil . Queríamos algo que transparecesse brasilidade só de olhar, como sacar logo de cara. A ideia foi dar um diferencial ainda maior para as peças através das estampas e cores. No uniforme da Seleção Brasileira feminina de handebol, utilizamos animais e toda a fauna brasileira com desenhos de araras com estampas de onça sobreposta”, afirmou a estilista."
1,1,Q0,clueweb22-pt0000-55-12305_1,2,9.3754,Anserini,"A presença marcante da m ata está no litoral brasileiro, em grande parte do Sudeste, do Sul e de alguns estados do Centro-Oeste e Nordeste. Estima-se que, quando os europeus aqui chegaram, em 1500, essa floresta ocupava 15% to território brasileiro, com uma área de 1.306.421 km². Com o passar dos anos e o intenso desmatamento causado pelas ações antrópicas, esse número é drasticamente menor, com pouco mais de 100 mil km² da vegetação original. Sua localização litorânea e de baixas e médias latitudes dá-lhe o caráter de uma floresta úmida, quente e com fauna e flora bastante desenvolvid a s. Como a ocupação brasileira ocorreu, de forma inicial, na parte leste do país, a mata foi o primeiro bioma a ser desmatado, o que explica sua baixa área. Localizada no litoral brasileiro, a Mata Atlântica foi o primeiro bioma desmatado do Brasil. Subdivisões da Mata Atlântica A Mata Atlântica tem algumas características bem semelhantes às da Floresta Amazônica. Essas características são referentes ao porte das árvores e ao desenvolvimento da fauna: ambos os biomas possuem árvores altas que podem chegar a mais de 30 metros de altura, com pequenas vegetações arbustivas que surgem à sombra delas."
2,1,Q0,clueweb22-pt0001-56-07770_0,3,9.1973,Anserini,"Peixe tambaqui: conheça a espécie e suas características Peixes Peixe tambaqui: conheça a espécie e suas características Seu nome científico é Colossoma macropomum. O peixe Tambaqui é popularmente conhecido como Pacu Vermelho. Este é o segundo maior peixe de escamas do Rio Amazonas, na região formada pelo regime do rio Solimões e pelo derretimento das geleiras dos Andes. Este é um peixe típico de água doce do Brasil. Habitante das matas inundadas, este peixe é encontrado em Mato Grosso, Goiás, São Paulo e no norte do país. São famosos por viver em águas pobres de minerais e, apesar disso, serem muito nutritivos e resistentes a doenças. Os animais compõem a biodiversidade da fauna das águas doces brasileiras, e atuam na cadeia alimentar como predadores e consumidores, o que infere à carne sabor peculiar e muito apreciado. Alimentação São onívoros, assim como algumas espécies terrestres como os marsupiais, eles comem de tudo. Têm hábitos diurnos, quando ficam ativos à procura de alimentos."
3,1,Q0,clueweb22-pt0001-71-09093_76,4,9.0899,Anserini,"biomas brasileiros (em quilômetros quadrados) 2% 5% 48% 10% 2% FABIO COLOMBINI 46 cm 8% 22% 49% Sagui-de-tufos-brancos, PI. Foto tirada em abril de 2015. Espécies animais no bioma 35% 8% 1,70 m Ema, GO. Foto tirada em agosto de 2017. DESCUBRA MAIS O Brasil é o país com maior número de espécies da fauna e da flora no mundo. Para 

output_filename=tests/bm25_1M_docs_000_999_sampled_50_queries_run.txt


In [32]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {results[0]}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 1M massiveweb filtered run and compute nDCG@10

In [34]:
results = prepare_run_format(BM25_1M_MASSIVEWEB, score_field='score', invert_score=False)

,query_id,Q0,passage_id,position,score,comment,passage
0,1,Q0,clueweb22-pt0000-05-02747_0,1,11.6703,Anserini,"BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA - Klima Naturali™ Klima Home Biomas BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA Tags Biomas Um bioma é um conjunto de tipos de vegetação que abrange grandes áreas contínuas, em escala regional, com flora e fauna similares, definida pelas condições físicas predominantes nas regiões. Esses aspectos climáticos, geográficos e litológicos (das rochas), por exemplo, fazem com que um bioma seja dotado de uma diversidade biológica singular, própria. No Brasil, os biomas existentes são (da maior extensão para a menor): a Amazônia, o cerrado, a Mata Atlântica, a Caatinga, o Pampa e o Pantanal. A seguir, conheça cada bioma do Brasil. Amazônia Extensão aproximada: 4.196.943 quilômetros quadrados"
1,1,Q0,clueweb22-pt0000-78-09747_0,2,10.1476,Anserini,"Onça-pintada - Escola Kids Qual matéria está procurando ? Ciências Home Ciências Onça-pintada Onça-pintada Que tal conhecer um pouco mais a respeito da incrível onça-pintada? Neste texto falaremos sobre o maior felino encontrado nas Américas. Conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. A onça-pintada é um dos símbolos da fauna brasileira A onça-pintada é um dos símbolos da fauna do Brasil e o maior felino encontrado nas Américas. Esse animal é um mamífero da ordem Carnivora e seu nome científico é Panthera onca. Leia também: Classificação dos mamíferos → Características A onça-pintada tem como característica marcante a sua pelagem, que varia do amarelo-claro ao castanho com algumas manchas pretas em forma de rosetas. Essas manchas funcionam como as nossas impressões digitais. Assim como as impressões em nossos dedos, as manchas da onça-pintada são únicas. As onças-pintadas apresentam peso que varia de 35 kg a 130 kg. Além disso, seu tamanho corporal é considerável. O comprimento total de uma onça varia de 1,7 m a 2,4 m."
2,1,Q0,clueweb22-pt0000-87-14633_2,3,9.9345,Anserini,"- Fauna e Flora A fauna da Amazônia é representada, principalmente, pela onça-pintada, boto-cor-de-rosa, arara-azul, tatu e cobras, como a jararaca. A flora da Amazônia apresenta cerca de 30 mil espécies. Entre as mais conhecidas, é possível destacar a vitória-régia, que é característica das matas de igapós. - Hidrografia A maior bacia hidrográfica do mundo é encontrada nesse bioma, detendo aproximadamente 20% da água doce do planeta, a Bacia Amazônica. O principal rio que drena esse bioma é o Rio Amazonas, o maior em volume de água do mundo. - Solo O solo do bioma Amazônia é normalmente arenoso. Possui uma camada de húmus proveniente da deposição da vegetação, frutos e restos de animais. Somente 14% do território é fértil e propício às atividades agrícolas. 2) Bioma Cerrado O Cerrado é considerado a savana brasileira. O Bioma Cerrado é conhecido como a savana brasileira, sendo o segundo maior bioma do país e também o segundo maior bioma da América do Sul. Constitui uma área de aproximadamente 2.036.448 km 2, representando cerca de 23,9% do território brasileiro. É um bioma que apresenta diferentes fitofisionomias (aspecto/feição de uma comunidade vegetal), devido aos muitos contatos biogeográficos com outros biomas."
3,1,Q0,clueweb22-pt0000-69-05954_7,4,9.9165,Anserini,"Aprenda sobre a Bacia do Tocantins-Araguaia, a maior bacia hidrográfica brasileira. Conheça melhor os seus rios principais e entenda a sua importância socioeconômica. Geografia do Brasil Bacias hidrográficas do Brasil Saiba mais sobre as bacias hidrográficas brasileiras. Entenda a importância delas para a natureza e para a sociedade em geral. Geografia do Brasil Biomas brasileiros Veja as características dos seis biomas brasileiros: Cerrado, Amazônia, Mata Atlântica, Pantanal, Pampa e Caatin

output_filename=tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt


In [35]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {results[0]}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare SPLADE + mT5 retrieval results in run format

In [36]:
results = prepare_run_format(SPLADE_MT5, score_field='score', invert_score=False)

,query,passage_id,score,query_id,colbertx_id,passage
0,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-61-07564_63,990,1,2899447,"A fauna da Floresta Amazônica tem a maior riqueza e diversidade do Brasil.[71] Dentre os macacos são destacados os saís, os macacos-de-cheiro, os barrigudos, os guaribas e os saguis. São também bastante numerosos os cachorros-do-mato, quatis, furões, jaguatiricas e preguiças. O peixe-boi, o boto-branco e o tucuxi são animais de exclusividade dessa região. O jacamim, o pavãozinho-do-pará, a arara-vermelha, o pavão-do-mato são aves típicas da Amazônia, enquanto que dentre os ofídios é importante mencionar, por serem grandes, as jiboias e as sucuris. Dentre os peixes da Amazônia são salientados o pirarucu, as raias, tucunaré, poraquê e o baiacu. São encontrados também jacarés, tartarugas e o surubim (este em abundância ao sul da Amazônia).[71] A fauna pantaneira também se destaca quanto à diversidade de espécies animais, sendo o bioma considerado como um dos maiores centros de reprodução das Américas. Aves como o jaburu, a garça, a arara-azul-grande e o biguá; mamíferos como a onça-pintada, o cervo-do-pantanal e o lobo-guará; répteis como a sucuri; e peixes como a piramboia, o dourado e o lambari representam algumas das espécies típicas do ambiente pantaneiro.[143][144] O muriqui-do-norte é um primata endêmico do Brasil e considerado criticamente em perigo"
1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-77-06884_1,970,1,3668516,"O tamanho do Brasil e a grande variedade de climas e características e suas regiões são o que torna possível existir no país um número tão alto de espécies. Dentre os biomas brasileiros, os maiores em biodiversidade são a Amazônia, a Mata Atlântica e o Cerrado. Este post é inspirado no livro infantil “Jornada pela Floresta” No livro personalizado Uma Jornada pela Floresta, as crianças são apresentadas de forma divertida e lúdica a vários animais típicos do Brasil: boto-cor-de-rosa, mico-leão-dourado, arara-azul, sucuri, ariranha e outros mais. Logo a seguir, você pode conferir curiosidades sobre cada espécie para compartilhar com os pequenos e aprender mais sobre esses animais que todos nós temos o dever de cuidar e proteger! Boto-cor-de-rosa O boto-cor-de-rosa nasce cinza e muda de cor com o passar do tempo. O boto-cor-de-rosa é um mamífero que vive na água. Ele é parente das baleias e dos golfinhos: a diferença é que ele não vive no mar, mas sim na água doce. Seu habitat são os rios amazônicos do Brasil, da Bolívia, da Venezuela, da Colômbia, do Equador e do Peru."
2,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-78-09747_0,940,1,3718313,"Onça-pintada - Escola Kids Qual matéria está procurando ? Ciências Home Ciências Onça-pintada Onça-pintada Que tal conhecer um pouco mais a respeito da incrível onça-pintada? Neste texto falaremos sobre o maior felino encontrado nas Américas. Conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. A onça-pintada é um dos símbolos da fauna brasileira A onça-pintada é um dos símbolos da fauna do Brasil e o maior felino encontrado nas Américas. Esse animal é um mamífero da ordem Carnivora e seu nome científico é Panthera onca. Leia também: Classificação dos mamíferos → Características A onça-pintada tem como característica marcante a sua pelagem, que varia do amarelo-claro ao castanho com algumas manchas pretas em forma de rosetas. Essas manchas funcionam como as nossas impressões digitais. Assim como as impressões em nossos dedos, as manchas da onça-pintada são únicas. As onças-pintadas apresentam peso que varia de 35 kg a 130 kg. Além disso, seu tamanho corporal é considerável. O comprimento total de uma onça varia de 1,7 m a 2,4 m."
3,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,870,1,3048261,"A exuberante fauna e flora brasileiras, relatadas por Pero Vaz de Caminha em sua carta à Coroa Portuguesa, está s

output_filename=tests/splade_mt5_docs_000_099_50_sampled_queries_run.txt


In [37]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {results[0]}

In [38]:
retrievers_results.append({'retriever':'SPLADE + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [39]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25 1M,0.3234
1,BM25 1M massiveweb,0.3203
2,SPLADE + mT5,0.7107
